<a href="https://colab.research.google.com/github/Xornotor/PPGEEC-CompEvolutiva-Atividades/blob/main/Avalia%C3%A7%C3%A3o_1B_Estrat%C3%A9gias_Evolutivas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Avaliação 1B - Computação Evolutiva e Meta-heurísticas**

**Docente:** Prof. Dr. Edmar Egídio Purcino de Souza

**Discentes:** André Paiva, Gabriel Lucas, Márcio Barros e Shaísta Câmara

## 1 - Inicialização e definição de funções das estratégias evolutivas

### 1.1 - Importação de dependências e parametrização

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from plotly import graph_objects as go
from plotly import express as px
import matplotlib as mpl
import seaborn as sns

sns.set_style("darkgrid")
mpl.rcParams['figure.dpi'] = 150
np.random.seed(5)

In [ ]:
general_conditions = {
    # Parâmetros para estratégias evolutivas

    #------------------------------------------------------------------
    # Escolha a função de fitness a ser usada: "dropwave" ou "levi"
    "functionName": "dropwave",
    #"functionName": "levi",
    #------------------------------------------------------------------

    # Estratégia (1 = (Mu,Lambda), 2 = (Mu+Lambda), 3 = CMA-ES)
    "strategy": 2,

    # Quantidade de pais
    "muSize": 25,

    # Quantidade de filhos
    "lambdaSize": 100,

    # Quantidade máxima de gerações
    "generationCount": 100,

    # Tamanho inicial do passo
    "initialSigma": 2,

    # Variação geracional do passo
    "deltaSigma": 1/np.sqrt(120),

    # Progenitores a serem recombinados
    "recombParents": 10,

    # Estratégia de recombinação (0 = Desativado, 1 = Intermédia, 2 = Discreta)
    "recombType": 1,

    # Limites de valores para os genes
    "bounds": np.array([[-100, 100], [-100, 100]])
}

## 1.2 - Definição de função objetivo (fitness)

In [ ]:
def fitness(chromosome, conditions):
    x1 = chromosome[0]
    x2 = chromosome[1]
    if conditions["functionName"] == "dropwave":
        r = np.sqrt(x1**2 + x2**2)
        numerator = -(1 + np.cos(12 * r))
        denominator = 0.5 * (x1**2 + x2**2) + 2
        return numerator / denominator
    elif conditions["functionName"] == "levi":
        term1 = np.sin(3 * np.pi * x1)**2
        term2 = (x1 - 1)**2 * (1 + np.sin(3 * np.pi * x2)**2)
        term3 = (x2 - 1)**2 * (1 + np.sin(2 * np.pi * x2)**2)
        return term1 + term2 + term3
    else:
        raise ValueError("Função de fitness desconhecida.")

In [ ]:
# Plot interativo da função objetivo
def fitness_plot(conditions):
    if conditions["functionName"] == "dropwave":
        x = y = np.linspace(-5, 5, 500)
    if conditions["functionName"] == "levi":
        x = y = np.linspace(-10, 10, 500)

    chromosome = np.meshgrid(x, y)
    Z = fitness(chromosome, conditions)

    fig = go.Figure(data=[go.Surface(z=Z, x=x, y=y, colorscale='rainbow')])
    fig.update_layout(title=f"Função {conditions['functionName'].capitalize()}",
                    autosize=False, width=800, height=600,
                    margin=dict(l=65, r=50, b=65, t=90))
    fig.show()

### 1.3 - Funções auxiliares para as estratégias evolutivas

In [ ]:
# check if a point is within the bounds of the search
def in_bounds(point, bounds):
	# enumerate all dimensions of the point
	for d in range(len(bounds)):
		# check if out of bounds for this dimension
		if point[d] < bounds[d, 0] or point[d] > bounds[d, 1]:
			return False
	return True

In [ ]:
# FUNÇÕES DE PLOT

# Plot das curvas de convergência
def convergence_curve_individual(population_df, conditions):
    x = np.arange(0, len(population_df))
    y = population_df["Fitness"].to_numpy()

    fig = plt.figure()
    ax = plt.gca()

    plt.plot(x, y, color='Plum', linestyle='--', linewidth=2,
            marker='o', markeredgewidth=1, markerfacecolor='FireBrick',
            markeredgecolor='black', markersize=2)

    plt.xlabel('Geração')
    plt.ylabel('Função objetivo')
    plt.xticks(np.linspace(0, len(population_df), 8, dtype=int))

    # Mostra no título qual função está sendo usada
    plt.title(f'Curva de convergência - Função {conditions["functionName"].capitalize()}')

    if(conditions["functionName"] == "dropwave"):
        plt.axhline(y = -1, color ="red", linestyle ="-", zorder=1.5, linewidth=2)
    elif(conditions["functionName"] == "levi"):
        plt.axhline(y = 0, color ="red", linestyle ="-", zorder=1.5, linewidth=2)

    plt.grid(True)
    plt.show()
    plt.close(fig)

    # Identificar última mudança significativa (> 1e-4) no fitness
    threshold = 1e-4
    last_significant_idx = 0

    for i in range(1, len(y)):
        if abs(y[i] - y[i - 1]) > threshold:
            last_significant_idx = i

### 1.4 - Estratégias evolutivas

In [ ]:
def mu_lambda_es(conditions):

    mu = conditions["muSize"]
    lam = conditions["lambdaSize"]
    bounds = conditions["bounds"]

    sigma = conditions["initialSigma"]

    best, best_eval = None, np.inf

    # Acúmulo de dados ao longo das gerações
    best_dicts = []
    population_df = pd.DataFrame()

	# calculate the number of children per parent
    n_children = int(lam / mu)
	# initial population
    population = list()

    for _ in range(lam):
        candidate = None
        while candidate is None or not in_bounds(candidate, bounds):
            candidate = bounds[:, 0] + np.random.rand(len(bounds)) * (bounds[:, 1] - bounds[:, 0])
        population.append(candidate)
	# perform the search
    for epoch in range(conditions["generationCount"]):
        if(conditions["recombParents"] > 0 and conditions["recombType"] != 0):
            for i in range(conditions["recombParents"]):
                if(conditions["recombType"] == 1):
                    # Intermédia
                    random_pick = np.random.choice(len(population), conditions["recombParents"], replace=False)
                    recomb_individual = np.sum(np.array(population)[random_pick], axis=0)/conditions["recombParents"]
                elif(conditions["recombType"] == 2):
                    # Discreta
                    random_pick = np.random.choice(len(population), 2, replace=False)
                    random_gene_sel = np.random.rand()
                    if(random_gene_sel < 0.5):
                        recomb_individual = np.array([population[random_pick[0]][0],
                                                      population[random_pick[1]][1]])
                    else:
                        recomb_individual = np.array([population[random_pick[1]][0],
                                                      population[random_pick[0]][1]])
                else:
                    raise ValueError("Estratégia de recombinação desconhecida.")
                population.append(recomb_individual.tolist())
		# evaluate fitness for the population
        scores = [fitness(c, conditions) for c in population]
		# rank scores in ascending order
        ranks = np.argsort(np.argsort(scores))
		# select the indexes for the top mu ranked solutions
        selected = [i for i,_ in enumerate(ranks) if ranks[i] < mu]

        parents_dicts = []
        for i in selected:
            parent_dict = {"Generation": epoch, "x1": population[i][0], "x2": population[i][1], "Fitness": scores[i]}
            parents_dicts.append(parent_dict)

        population_df = pd.concat([population_df, pd.DataFrame(parents_dicts)])

		# create children from parents
        children = list()

        for i in selected:
            # check if this parent is the best solution ever seen
            if scores[i] < best_eval:
                best, best_eval = population[i], scores[i]
                print('%d, Best: f(%s) = %.5f' % (epoch, best, best_eval))

			# create children for parent
            for _ in range(n_children):
                child = None
                while child is None or not in_bounds(child, bounds):
                    child = population[i] + np.random.randn(len(bounds)) * sigma
                children.append(child)

            # keep the parent if (Mu+Lambda)
            if(conditions["strategy"]) == 2:
                children.append(population[i])

        # Save best candidate of the generation
        best_dicts.append({"Generation": epoch, "x1": best[0], "x2": best[1], "Fitness": best_eval})

		# replace population with children
        population = children

        # Variação de sigma
        sigma *= np.exp((np.random.normal(0, 1) * (conditions["deltaSigma"]**2)))

    best_df = pd.DataFrame(best_dicts)

    return best, best_eval, best_df, population_df

## 2 - Testes de Estratégias Evolutivas: Execução Única

In [ ]:
fitness_plot(general_conditions)

In [ ]:
best, best_eval, best_df, population_df = mu_lambda_es(general_conditions)

In [ ]:
px.scatter(population_df, "x1", "x2", "Fitness", animation_frame="Generation",
           width=800, height=600, color_continuous_scale='bluered',
           title=f"Função {general_conditions['functionName'].capitalize()} - Evolução das Gerações")

In [ ]:
convergence_curve_individual(best_df, general_conditions)